In [1]:
from cyvcf2 import VCF
import pandas as pd
import time
import multiprocessing
from datetime import date

In [2]:
def add_columns(df):
    data_1 = df.copy()
    data_1['REF'] = '.'
    data_1['ALT'] = '.'
    data_1['GENOTYPE'] = '.'
    
    return data_1

In [3]:
def chunk(L,nchunks):
    L2 = list()
    j = round(len(L)/nchunks)
    chunk_size = j
    i = 0
    while i < len(L):
        chunk = L[i:j]
        L2.append(chunk)
        i = j
        j += chunk_size
        if(j>len(L)):
            j = len(L)
    return L2

In [4]:
def worker(input_chunk):
    ref_alt_library = VCF(myinputvcf)
    
    lines = []
    
    for chrom_pos in input_chunk:
        chromie = chrom_pos[0]
        base_position = chrom_pos[1]
        target_row = data[(data['chromosome'] == chromie) & (data['position'] == base_position)]
        
        allele1 = target_row['allele1']
        allele1 = str(allele1.values)
        allele1 = allele1.replace("['","")
        allele1 = allele1.replace("']","")
        
        allele2 = target_row['allele2\r']
        allele2 = str(allele2.values)
        allele2 = allele2.replace("['","")
        allele2 = allele2.replace("']","")
        
        position = str(chromie) + ":" + str(base_position) + "-" + str(base_position)
        
        for record in ref_alt_library(position):
            read = str(record).split()
            chrom, bp, rsid, REF, ALT = read[0], read[1], read[2], read[3], read[4]
            
            
            if str(allele1) == str(REF) and str(allele2) == str(REF):
                genotypes = '0/0'
            elif str(allele1) == str(REF) and str(allele2) == str(ALT):
                genotypes = '0/1'
            elif str(allele1) == str(ALT) and str(allele2) == str(REF):
                genotypes = '1/0'
            elif str(allele1) == str(ALT) and str(allele2) == str(ALT):
                genotypes = '1/1'
            else:
                genotypes = '.'
                
            line =[chrom, bp, rsid, REF, ALT, '.', '.', '.', 'GT', genotypes]
            
            lines.append(line)
            
    return lines   

In [7]:
def test_get_ref_alt(df):
    
    results = []
    
    data_2 = add_columns(df)

    for currChrom in range(1,23):
        print('We are starting chromosome' + str(currChrom))
        global myinputvcf
        myinputvcf = '/gpfs/group/torkamani/shaun/mygenerank-pipeline_beta/conversion/1000G_HG00096/ALL.chr'+str(currChrom)+'.phase3.HG00096.vcf.gz'

        coll = data[data['chromosome'] == currChrom]
        target_coll = coll.loc[:, 'chromosome' : 'position']
        target_list = target_coll.values.tolist()

        chunks = chunk(target_list, ncores)
        pool = multiprocessing.Pool(ncores)
        temp_results = pool.map(worker, chunks)
        
        pool.close()
        pool.join()

        temp_results = [item for sublist in temp_results for item in sublist]
        results += temp_results
        
    return results

In [9]:
start = time.time()
print('began')

myinputtxt = 'AncestryDNA2.txt'
original_file = pd.read_csv(myinputtxt, sep = '\t', lineterminator='\n', comment='#')
data = (original_file[original_file["chromosome"] <= 22])

ncores = multiprocessing.cpu_count()


sub_dict = {}
myinputvcf = ""

outputfilename = myinputtxt.strip('.txt') + '.vcf'
with open(outputfilename, 'w') as outputfile:

    today = date.today().strftime('%Y%m%d')
    header = ['##fileformat=VCFv4.2',
              '##fileDate=' + today,
              '##source=conversion_from_Ancestry_user_download',
              '##reference=1000Genomes',
              '##INFO=<ID=ID,Number=1,Type=Integer,Description="N/A">',
              '##FORMAT=<ID=GT,Number=1,Type=String,Description="N/A">',
              '##FORMAT=<ID=NA,Number=1,Type=String,Description="N/A">']
    header = "\n".join(header)



    finaldata = test_get_ref_alt(data)
    end_data = pd.DataFrame(finaldata, columns = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'GENOTYPE'])

    outputfile.write(header + '\n')
    outputfile.write(end_data.to_string())


end = time.time()

print('Conversion Time:')
print(end - start)
    
    

began
We are starting chromosome1
We are starting chromosome2
We are starting chromosome3
We are starting chromosome4
We are starting chromosome5
We are starting chromosome6
We are starting chromosome7
We are starting chromosome8
We are starting chromosome9
We are starting chromosome10
We are starting chromosome11
We are starting chromosome12
We are starting chromosome13
We are starting chromosome14
We are starting chromosome15
We are starting chromosome16
We are starting chromosome17
We are starting chromosome18
We are starting chromosome19
We are starting chromosome20
We are starting chromosome21
We are starting chromosome22
Conversion Time:
249.01769852638245
